# Model Comparison and Ensemble Modelling

## 1. Introduction

### **Objective:**
To assess and compare the performance of multiple SDMs (GLM, GAM, RF, XGBoost, MaxEnt) and explore ensemble modelling techniques to improve species distribution predictions.

### **Rationale:**
Comparing different models allows us to identify strengths and weaknesses in their predictive capabilities. Ensemble modelling combines multiple models to leverage their individual strengths, often leading to improved accuracy and robustness in predictions (Meller et al., 2014). 

In [2]:
import os
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_auc_score, roc_curve, precision_recall_curve, 
    precision_score, recall_score, f1_score, confusion_matrix
)


## 2. Data Preparation

### **Steps:**

#### 1. Load Test Predictions:
- Import the test prediction results from each model for all species.

#### 2. Ensure Consistency:
- Verify that all datasets have consistent formatting, with aligned columns for true labels and predicted probabilities.

### **Rationale:**
Consistent data formatting is crucial for accurate performance evaluation and comparison across models.

### 2.1 Loading GLM Files

In [3]:
import os
import joblib
import pandas as pd

# Define paths to final GLM models and metrics
final_glm_dir = "C:/GIS_Course/MScThesis-MaviSantarelli/results/Models/Final_GLM/"
model_dir = os.path.join(final_glm_dir, "Models")  # Path to final models

# Species list
species_list = ["Bufo bufo", "Rana temporaria", "Lissotriton helveticus"]
glm_versions = ["Lasso", "Ridge"]  # Regularised models

# Dictionary to store models and metrics
glm_models = {}
glm_metrics = {}

# Load GLM models and metrics
for species in species_list:
    glm_models[species] = {}
    glm_metrics[species] = {}

    for glm_type in glm_versions:
        model_path = os.path.join(model_dir, f"{species}_GLM_{glm_type}_Threshold_0.3_Model.pkl")
        metrics_path = os.path.join(final_glm_dir, f"{species}_Threshold_0.3_Metrics.txt")
        
        if os.path.exists(model_path):
            glm_models[species][glm_type] = joblib.load(model_path)
            print(f"Loaded {glm_type} model for {species}")
        else:
            print(f"⚠️ Warning: Model not found for {species} - {glm_type}")

        if os.path.exists(metrics_path):
            with open(metrics_path, "r") as f:
                glm_metrics[species][glm_type] = f.read()
            print(f"Loaded metrics for {species} - {glm_type}")
        else:
            print(f"⚠️ Warning: Metrics file not found for {species} - {glm_type}")

print("✅ Final GLM models and metrics loaded successfully!")


Loaded Lasso model for Bufo bufo
Loaded metrics for Bufo bufo - Lasso
Loaded Ridge model for Bufo bufo
Loaded metrics for Bufo bufo - Ridge
Loaded Lasso model for Rana temporaria
Loaded metrics for Rana temporaria - Lasso
Loaded Ridge model for Rana temporaria
Loaded metrics for Rana temporaria - Ridge
Loaded Lasso model for Lissotriton helveticus
Loaded metrics for Lissotriton helveticus - Lasso
Loaded Ridge model for Lissotriton helveticus
Loaded metrics for Lissotriton helveticus - Ridge
✅ Final GLM models and metrics loaded successfully!


### 2.2 Loading GAM Files

In [6]:
import os
import joblib
import pandas as pd

# Define base directory for final GAM models and metrics
gam_results_dir = "C:/GIS_Course/MScThesis-MaviSantarelli/results/Models/Final_GAM"

# Define species list (keeping spaces instead of converting to underscores)
species_list = ["Bufo bufo", "Rana temporaria", "Lissotriton helveticus"]

# Dictionary to store loaded models and metrics
gam_models = {}
gam_metrics = {}

# Load models and metrics
for species in species_list:
    # Use original filenames without changing spaces to underscores
    model_path = os.path.join(gam_results_dir, f"{species}_GAM_Model_CV.pkl")
    metrics_path = os.path.join(gam_results_dir, f"{species}_GAM_Test_Metrics.txt")

    # Load the model
    if os.path.exists(model_path):
        gam_models[species] = joblib.load(model_path)
        print(f" Loaded GAM model for {species}")
    else:
        print(f"⚠️ GAM model file missing for {species}: {model_path}")

    # Load the metrics
    if os.path.exists(metrics_path):
        with open(metrics_path, "r") as f:
            gam_metrics[species] = f.read()
        print(f" Loaded GAM metrics for {species}")
    else:
        print(f"⚠️ GAM metrics file missing for {species}: {metrics_path}")

print("\n✅ Final GAM models and metrics loaded successfully!")


 Loaded GAM model for Bufo bufo
 Loaded GAM metrics for Bufo bufo
 Loaded GAM model for Rana temporaria
 Loaded GAM metrics for Rana temporaria
 Loaded GAM model for Lissotriton helveticus
 Loaded GAM metrics for Lissotriton helveticus

✅ Final GAM models and metrics loaded successfully!


### 2.3 Loading RF Files

In [7]:
import os
import joblib
import pandas as pd

# Define base directory where the models are stored
base_output_dir = "C:/GIS_Course/MScThesis-MaviSantarelli/results/Models/RandomForest"

# List of species to load models for
species_list = ["Bufo_bufo", "Rana_temporaria", "Lissotriton_helveticus"]

# Dictionary to store loaded models and metrics
rf_models = {}
rf_metrics = {}
rf_predictions = {}

for species in species_list:
    species_dir = os.path.join(base_output_dir, species)
    print(f"Loading Random Forest model and metrics for {species}...")
    
    # Load the final trained model
    model_path = os.path.join(species_dir, "RandomForest_Model.pkl")
    if os.path.exists(model_path):
        rf_models[species] = joblib.load(model_path)
        print(f"Loaded model for {species}")
    else:
        print(f"⚠️ Model file missing for {species}: {model_path}")
    
    # Load metrics
    metrics_path = os.path.join(species_dir, "RandomForest_Metrics.txt")
    if os.path.exists(metrics_path):
        with open(metrics_path, "r") as f:
            rf_metrics[species] = f.read()
        print(f"Loaded metrics for {species}")
    else:
        print(f"⚠️ Metrics file missing for {species}: {metrics_path}")
    
    # Load test predictions
    predictions_path = os.path.join(species_dir, "Test_Predictions.csv")
    if os.path.exists(predictions_path):
        rf_predictions[species] = pd.read_csv(predictions_path)
        print(f"Loaded test predictions for {species}")
    else:
        print(f"⚠️ Test predictions file missing for {species}: {predictions_path}")

print("\n✅ Random Forest models and metrics loaded successfully!")


Loading Random Forest model and metrics for Bufo_bufo...
Loaded model for Bufo_bufo
Loaded metrics for Bufo_bufo
Loaded test predictions for Bufo_bufo
Loading Random Forest model and metrics for Rana_temporaria...
Loaded model for Rana_temporaria
Loaded metrics for Rana_temporaria
Loaded test predictions for Rana_temporaria
Loading Random Forest model and metrics for Lissotriton_helveticus...
Loaded model for Lissotriton_helveticus
Loaded metrics for Lissotriton_helveticus
Loaded test predictions for Lissotriton_helveticus

✅ Random Forest models and metrics loaded successfully!


### 2.4 Loading XGBoost Files

In [8]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Base directory for XGBoost models
base_output_dir = "C:/GIS_Course/MScThesis-MaviSantarelli/results/Models"
xgb_output_dir = os.path.join(base_output_dir, "XGBoost")

# List of species
species_list = ["Bufo_bufo", "Rana_temporaria", "Lissotriton_helveticus"]

# Dictionary to store loaded models and metrics
xgb_models = {}
xgb_metrics = {}
xgb_test_predictions = {}

# Load models, metrics, and test predictions
for species in species_list:
    print(f"Loading XGBoost model and metrics for {species}...")

    species_output_dir = os.path.join(xgb_output_dir, species)
    
    # Load final model
    model_path = os.path.join(species_output_dir, "XGBoost_Model.pkl")
    xgb_models[species] = joblib.load(model_path)
    print(f"  Loaded model for {species}")

    # Load metrics
    metrics_path = os.path.join(species_output_dir, "XGBoost_Metrics.txt")
    with open(metrics_path, "r") as f:
        metrics = f.readlines()
    xgb_metrics[species] = {line.split(":")[0].strip(): float(line.split(":")[1].strip()) for line in metrics}
    print(f"  Loaded metrics for {species}")

    # Load test predictions
    test_predictions_path = os.path.join(species_output_dir, "Aggregated_Test_Predictions.csv")
    xgb_test_predictions[species] = pd.read_csv(test_predictions_path)
    print(f"  Loaded test predictions for {species}")

print("\n✅ XGBoost models and metrics loaded successfully!")


Loading XGBoost model and metrics for Bufo_bufo...
  Loaded model for Bufo_bufo
  Loaded metrics for Bufo_bufo
  Loaded test predictions for Bufo_bufo
Loading XGBoost model and metrics for Rana_temporaria...
  Loaded model for Rana_temporaria
  Loaded metrics for Rana_temporaria
  Loaded test predictions for Rana_temporaria
Loading XGBoost model and metrics for Lissotriton_helveticus...
  Loaded model for Lissotriton_helveticus
  Loaded metrics for Lissotriton_helveticus
  Loaded test predictions for Lissotriton_helveticus

✅ XGBoost models and metrics loaded successfully!


### 3.5 Loading Maxent Files  

In [11]:
import os
import pandas as pd

# Define the correct directory path
base_output_dir = r"C:\GIS_Course\MScThesis-MaviSantarelli\results\Models\Maxent"

# Define file paths using os.path.join
maxent_files = {
    "Bufo bufo": {
        "test_predictions": os.path.join(base_output_dir, "Maxent_Bufo_bufo_TestPredictions.csv"),
        "model": os.path.join(base_output_dir, "Maxent_Bufo_bufo.rds"),
    },
    "Rana temporaria": {
        "test_predictions": os.path.join(base_output_dir, "Maxent_Rana_temporaria_TestPredictions.csv"),
        "model": os.path.join(base_output_dir, "Maxent_Rana_temporaria.rds"),
    },
    "Lissotriton helveticus": {
        "test_predictions": os.path.join(base_output_dir, "Maxent_Lissotriton_helveticus_TestPredictions.csv"),
        "model": os.path.join(base_output_dir, "Maxent_Lissotriton_helveticus.rds"),
    },
}

# Check if files exist before loading
for species, paths in maxent_files.items():
    if os.path.exists(paths["test_predictions"]):
        print(f"✅ Found: {paths['test_predictions']}")
    else:
        print(f"❌ Missing: {paths['test_predictions']}")

# Now try loading the files
maxent_test_predictions = {}
for species, paths in maxent_files.items():
    maxent_test_predictions[species] = pd.read_csv(paths["test_predictions"])

print("\n✅ MaxEnt test predictions loaded successfully!")

✅ Found: C:\GIS_Course\MScThesis-MaviSantarelli\results\Models\Maxent\Maxent_Bufo_bufo_TestPredictions.csv
✅ Found: C:\GIS_Course\MScThesis-MaviSantarelli\results\Models\Maxent\Maxent_Rana_temporaria_TestPredictions.csv
✅ Found: C:\GIS_Course\MScThesis-MaviSantarelli\results\Models\Maxent\Maxent_Lissotriton_helveticus_TestPredictions.csv

✅ MaxEnt test predictions loaded successfully!


In [12]:
maxent_evaluation_path = os.path.normpath(r"C:\GIS_Course\MScThesis-MaviSantarelli\results\Models\Maxent\Maxent_Model_Evaluation.csv")
maxent_evaluation_df = pd.read_csv(maxent_evaluation_path)
print("✅ MaxEnt model evaluation loaded successfully!")


✅ MaxEnt model evaluation loaded successfully!


## 3. Model Performance Evaluation

### **Metrics used in Evaluation**:

- **Area Under the Receiver Operating Characteristic Curve (AUC-ROC)**: Measures the ability of the model to distinguish between classes.
- **Confusion Matrix:** Provides a summary of prediction results, showing true positives, true negatives, false positives, and false negatives.
- **Precision, Recall, and F1-Score:** Evaluate the accuracy of positive predictions, the ability to find all positive instances, and the balance between precision and recall, respectively.

### **Rationale:**
These metrics offer a comprehensive view of each model's performance, highlighting different aspects of predictive accuracy and error rates.

## 4. Visualisation of Model Performance

### **Steps:**
#### 1. Plot ROC Curves:
- Visualise the trade-off between true positive and false positive rates for each model.

#### 2. Compare Performance Metrics:
- Create bar plots or tables to compare precision, recall, and F1-scores across models.

### **Rationale:**
Visual representations facilitate intuitive comparisons and help identify models that perform well across various metrics.

## 5. Ensemble Modelling

### **Techniques to Explore:**

- **Weighted Averaging:** Combine model predictions by assigning weights proportional to their performance metrics.
- **Majority Voting:** For classification tasks, predict the class that receives the majority vote from individual models.

### **Rationale:**
Ensemble methods aim to harness the strengths of multiple models, often resulting in improved predictive performance and reduced variance (Meller et al., 2014). 

## 6. Evaluation of Ensemble Models

### **Steps:**
- **Compute Performance Metrics:** Assess the ensemble models using the same metrics as individual models (AUC-ROC, precision, recall, F1-score).
- **Compare to Individual Models:** Determine if the ensemble models outperform individual models.

### **Rationale:**
Evaluating ensemble models against individual models helps in understanding the added value of ensembling techniques.

## 7. Conclusion and Next Steps


## References

Meller, L., Cabeza, M., Pironon, S., Barbet-Massin, M., Maiorano, L., Georges, D., & Thuiller, W. (2014). Ensemble distribution models in conservation prioritization: From consensus predictions to consensus reserve networks. *Diversity and Distributions*, 20(3), 309–321. https://doi.org/10.1111/ddi.12162